In [1]:
from sionna.rt import load_scene, PlanarArray, Receiver, Transmitter, ITURadioMaterial, PathSolver
import numpy as np
import mitsuba as mi


In [2]:
scene = load_scene(filename="minis/row1_col4.xml")

In [3]:
scene.frequency # Defaults to 3.5GHz

mat = ITURadioMaterial(name='concrete', itu_type='concrete', thickness=0.1, scattering_coefficient=0.0, xpd_coefficient=0.0)

# Set all objects to concrete for now
# for obj in scene.objects.values():
#     obj.radio_material = mat

# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=1,
                          num_cols=1,
                          vertical_spacing=1,
                          horizontal_spacing=1,
                          polarization="V",
                          pattern="iso")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                          num_cols=1,
                          vertical_spacing=0.5,
                          horizontal_spacing=0.5,
                          polarization="V",
                          pattern="iso")

tx = Transmitter(name="tx",
              position=[160,100,40],
              orientation=[0,0,0])
scene.add(tx)

# # Create transmitter
# rx = Receiver(name="rx",
#               position=[160,300,40],
#               orientation=[0,0,0])
# scene.add(rx)

In [4]:
rx_points = []
spacing = 50
min = (0, 0, 0)
max = (500, 500, 1000)

x_start, y_start, _ = min
x_end, y_end, _ = max
xx = np.linspace(x_start, x_end, spacing)
yy = np.linspace(y_start, y_end, spacing)
ray_pos = np.array([(x, y, 1000) for x in xx for y in yy])

z_rays = mi.Ray3f(o=ray_pos.T, d=(0, 0, -1))
z_hits = scene.mi_scene.ray_intersect(z_rays)

for z_hit, valid in zip(z_hits.p.numpy().T, z_hits.is_valid()):
    if (not valid):
        continue
    start = int(z_hit[2] + 10)

    for i in range(start, 200, spacing):
        rx_points.append(np.array([z_hit[0], z_hit[1], i]))

# Include points that did not have a valid hitpoint due to lack of floor right now
for i, v in enumerate(z_hits.is_valid()):
    if (not v):
        r = ray_pos[i]
        for i in range(0, 200, spacing):
            rx_points.append(np.array([r[0], r[1], i]))

In [5]:
# Create receivers
for rx_i in range(len(rx_points)):
#     print(rx_points[rx_i])
    rx = Receiver(name=f"rx_{rx_i}",
            position=rx_points[rx_i],
            orientation=[0,0,0])
    scene.add(rx)

In [6]:
scene.preview()

Renderer(camera=PerspectiveCamera(aspect=1.31, children=(DirectionalLight(intensity=0.25, position=(0.0, 0.0, …

In [16]:
# This cell is lowkey pretty illegal but here's to being easier to implement
## THIS REQUIRES persist=True in call self._add_child(mesh, pmin, pmax, persist=True) in _plot_lines (preview.py)

starts = np.array([[0,0,0],
                   [1, 1, 1]])
ends = np.array([[5,5,5],
                 [-100, -100, 0]])
colors = np.array([1.0, 0.0, 0.0], dtype=np.float32)
colors = np.tile(colors, (starts.shape[0], 1))
print(colors)
print(colors.dtype)
width = np.array(1.0)
scene._preview_widget._plot_lines(starts, ends, colors, width)


[[1. 0. 0.]
 [1. 0. 0.]]
float32


In [8]:
scene.preview()
# scene.preview(paths=paths)


Renderer(camera=PerspectiveCamera(aspect=1.31, children=(DirectionalLight(intensity=0.25, matrixWorldNeedsUpda…

In [9]:
# p_solver  = PathSolver()
# paths = p_solver(scene=scene,
#                 max_depth=7,
#                 los=False,
#                 specular_reflection=True,
#                 diffuse_reflection=False,
#                 refraction=False,
#                 synthetic_array=False,
#                 seed=41)